# Other Training enemies

Bisher hat TD-Gammon im Selfplay nur gegen sich selber gelernt. Was passiert aber, wenn TD-Gammon zB. gegen singleton spielt und dabei lernt?

In [36]:
from NeuralNetModel import NeuralNetModel
from FasterBackgammon import Game
import random
import numpy as np
import Player

class NoSelfplayModel(NeuralNetModel):
    
    def __init__(self, sess, enemyPlayer, restore=False):
        self.enemyPlayer = enemyPlayer
        NeuralNetModel.__init__(self, sess, restore)
    
    def create_training_op(self):
        """
            TD-Gammon
        """
        
        #lambda, leider ohne b da dies ein Schlüsselwort ist
        lamda = tf.constant(0.7)

        #learning rate 
        alpha = tf.constant(0.1)

        #Global_step wird bei jeden aufruf von sess.run um 1 erhöht
        global_step_op = self.global_step.assign_add(1)

        # get gradients of output V wrt trainable variables (weights and biases)
        tvars = tf.trainable_variables()
        grads = tf.gradients(self.V, tvars)

        #Alle Variablen werden angepasst und mittels eligebility traces wird er Wert
        #der einzelnen Gewichte angepasst um gute bzw. schlechte Entscheidungen zu reflektieren
        apply_gradients = []

        for grad, var in zip(grads, tvars):
            # e-> = lambda * e-> + <grad of output w.r.t weights>
            trace = tf.Variable(tf.zeros(grad.get_shape()), trainable=False, name='trace')
            trace_op = trace.assign((lamda * trace) + grad)

            # grad with trace = alpha * delta * e
            grad_trace = alpha * self.delta_op * trace_op

            grad_apply = var.assign_add(grad_trace)
            apply_gradients.append(grad_apply)

        #Den global_step mitzählen lassen
        apply_gradients.append(global_step_op)

        #Alle Gradientenoperationen in einer train Operation zusammenfassen
        return tf.group(*apply_gradients, name='train')
    
    def train(self, games, validation_interval, test_games=100):
        #Selbsttraining, Modell vs Modell
        players = [Player.ModelPlayer('black', self), self.enemyPlayer]

        for i in range(games):
            #Immer wieder zwischendurch testen und den Fortschritt speichern
            if i != 0 and i % validation_interval == 0:
                self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step)
                print("Progress saved!")
                self.test(games = test_games)

            #Spiel initialisieren
            game = Game()
            player_num = random.randint(0, 1)
            
            x = game.extractFeatures(players[player_num].player)

            #Spiel spielen bis es einen Sieger gibt
            game_step = 0
            while not game.get_winner():
                game.next_step(players[player_num], player_num)
                player_num = (player_num + 1) % 2

                #Das Modell mit jeden Schritt im Spiel trainieren
                x_next = game.extractFeatures(players[player_num].player)
                V_next = self.get_output(x_next)
                
                self.sess.run([self.train_op, self.delta_op], feed_dict={ self.x: x, self.V_next: V_next })

                x = x_next
                game_step += 1

            #Gewinner ermitteln
            winner = 0 if game.get_winner() == game.players[0] else 1

            #Zu guter letzt reinforcement learning: Dem Modell noch eine "Belohnung" geben, wenn es gewonnen hat
            _, global_step = self.sess.run([
                self.train_op,
                self.global_step,
            ], feed_dict={ self.x: x, self.V_next: np.array([[winner]], dtype='float') })

            #Konsolenausgabe hübsch aufbereiten
            print("Game %d/%d (Winner: %s) in %d turns" % (i, games, players[winner].player, game_step))
        #Am Ende noch mal speichern und 100 testen!
        self.saver.save(self.sess, self.checkpoint_path + 'checkpoint.ckpt', global_step=global_step) 
        self.test(games = test_games)
        
    def get_name(self):
        return "NotSelfplayModel[" + self.enemyPlayer.get_name() + "]"

In [24]:
import tensorflow as tf

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = NoSelfplayModel(sess, Player.ValuePlayer('white', Player.singleton), restore=False)
    model.train(10000,1000)

Game 0/10000 (Winner: black) in 46 turns
Game 1/10000 (Winner: white) in 42 turns
Game 2/10000 (Winner: white) in 52 turns
Game 3/10000 (Winner: black) in 56 turns
Game 4/10000 (Winner: white) in 44 turns
Game 5/10000 (Winner: white) in 53 turns
Game 6/10000 (Winner: white) in 57 turns
Game 7/10000 (Winner: white) in 55 turns
Game 8/10000 (Winner: white) in 51 turns
Game 9/10000 (Winner: black) in 55 turns
Game 10/10000 (Winner: white) in 48 turns
Game 11/10000 (Winner: white) in 51 turns
Game 12/10000 (Winner: white) in 42 turns
Game 13/10000 (Winner: black) in 52 turns
Game 14/10000 (Winner: white) in 83 turns
Game 15/10000 (Winner: white) in 72 turns
Game 16/10000 (Winner: black) in 56 turns
Game 17/10000 (Winner: white) in 47 turns
Game 18/10000 (Winner: black) in 61 turns
Game 19/10000 (Winner: white) in 48 turns
Game 20/10000 (Winner: white) in 50 turns
Game 21/10000 (Winner: black) in 51 turns
Game 22/10000 (Winner: white) in 40 turns
Game 23/10000 (Winner: white) in 49 turns
Ga

Game 195/10000 (Winner: white) in 63 turns
Game 196/10000 (Winner: white) in 40 turns
Game 197/10000 (Winner: white) in 62 turns
Game 198/10000 (Winner: white) in 54 turns
Game 199/10000 (Winner: white) in 59 turns
Game 200/10000 (Winner: white) in 45 turns
Game 201/10000 (Winner: black) in 45 turns
Game 202/10000 (Winner: black) in 53 turns
Game 203/10000 (Winner: white) in 50 turns
Game 204/10000 (Winner: white) in 53 turns
Game 205/10000 (Winner: white) in 50 turns
Game 206/10000 (Winner: white) in 51 turns
Game 207/10000 (Winner: white) in 62 turns
Game 208/10000 (Winner: black) in 51 turns
Game 209/10000 (Winner: black) in 45 turns
Game 210/10000 (Winner: white) in 63 turns
Game 211/10000 (Winner: white) in 42 turns
Game 212/10000 (Winner: white) in 47 turns
Game 213/10000 (Winner: black) in 43 turns
Game 214/10000 (Winner: black) in 44 turns
Game 215/10000 (Winner: white) in 55 turns
Game 216/10000 (Winner: white) in 59 turns
Game 217/10000 (Winner: white) in 48 turns
Game 218/10

Game 387/10000 (Winner: white) in 60 turns
Game 388/10000 (Winner: white) in 53 turns
Game 389/10000 (Winner: black) in 47 turns
Game 390/10000 (Winner: black) in 52 turns
Game 391/10000 (Winner: black) in 43 turns
Game 392/10000 (Winner: white) in 52 turns
Game 393/10000 (Winner: black) in 48 turns
Game 394/10000 (Winner: black) in 45 turns
Game 395/10000 (Winner: white) in 57 turns
Game 396/10000 (Winner: black) in 69 turns
Game 397/10000 (Winner: white) in 41 turns
Game 398/10000 (Winner: white) in 52 turns
Game 399/10000 (Winner: white) in 46 turns
Game 400/10000 (Winner: white) in 47 turns
Game 401/10000 (Winner: white) in 50 turns
Game 402/10000 (Winner: black) in 59 turns
Game 403/10000 (Winner: white) in 47 turns
Game 404/10000 (Winner: black) in 47 turns
Game 405/10000 (Winner: black) in 48 turns
Game 406/10000 (Winner: white) in 44 turns
Game 407/10000 (Winner: black) in 60 turns
Game 408/10000 (Winner: white) in 44 turns
Game 409/10000 (Winner: black) in 48 turns
Game 410/10

Game 578/10000 (Winner: white) in 46 turns
Game 579/10000 (Winner: white) in 41 turns
Game 580/10000 (Winner: black) in 48 turns
Game 581/10000 (Winner: white) in 45 turns
Game 582/10000 (Winner: white) in 43 turns
Game 583/10000 (Winner: black) in 43 turns
Game 584/10000 (Winner: white) in 48 turns
Game 585/10000 (Winner: white) in 50 turns
Game 586/10000 (Winner: white) in 50 turns
Game 587/10000 (Winner: black) in 47 turns
Game 588/10000 (Winner: white) in 46 turns
Game 589/10000 (Winner: white) in 48 turns
Game 590/10000 (Winner: black) in 37 turns
Game 591/10000 (Winner: black) in 47 turns
Game 592/10000 (Winner: white) in 52 turns
Game 593/10000 (Winner: white) in 49 turns
Game 594/10000 (Winner: white) in 43 turns
Game 595/10000 (Winner: black) in 45 turns
Game 596/10000 (Winner: white) in 47 turns
Game 597/10000 (Winner: white) in 42 turns
Game 598/10000 (Winner: white) in 48 turns
Game 599/10000 (Winner: white) in 55 turns
Game 600/10000 (Winner: black) in 47 turns
Game 601/10

Game 769/10000 (Winner: black) in 43 turns
Game 770/10000 (Winner: black) in 50 turns
Game 771/10000 (Winner: black) in 48 turns
Game 772/10000 (Winner: white) in 55 turns
Game 773/10000 (Winner: white) in 52 turns
Game 774/10000 (Winner: white) in 46 turns
Game 775/10000 (Winner: white) in 50 turns
Game 776/10000 (Winner: white) in 51 turns
Game 777/10000 (Winner: black) in 52 turns
Game 778/10000 (Winner: white) in 64 turns
Game 779/10000 (Winner: white) in 56 turns
Game 780/10000 (Winner: white) in 45 turns
Game 781/10000 (Winner: white) in 50 turns
Game 782/10000 (Winner: white) in 46 turns
Game 783/10000 (Winner: black) in 52 turns
Game 784/10000 (Winner: white) in 53 turns
Game 785/10000 (Winner: black) in 47 turns
Game 786/10000 (Winner: white) in 54 turns
Game 787/10000 (Winner: white) in 49 turns
Game 788/10000 (Winner: white) in 44 turns
Game 789/10000 (Winner: white) in 44 turns
Game 790/10000 (Winner: white) in 45 turns
Game 791/10000 (Winner: white) in 42 turns
Game 792/10

...

Game 9487/10000 (Winner: black) in 38 turns
Game 9488/10000 (Winner: white) in 51 turns
Game 9489/10000 (Winner: black) in 49 turns
Game 9490/10000 (Winner: black) in 39 turns
Game 9491/10000 (Winner: white) in 66 turns
Game 9492/10000 (Winner: white) in 58 turns
Game 9493/10000 (Winner: black) in 45 turns
Game 9494/10000 (Winner: black) in 47 turns
Game 9495/10000 (Winner: black) in 41 turns
Game 9496/10000 (Winner: black) in 53 turns
Game 9497/10000 (Winner: white) in 54 turns
Game 9498/10000 (Winner: white) in 45 turns
Game 9499/10000 (Winner: black) in 56 turns
Game 9500/10000 (Winner: white) in 54 turns
Game 9501/10000 (Winner: black) in 60 turns
Game 9502/10000 (Winner: white) in 64 turns
Game 9503/10000 (Winner: white) in 56 turns
Game 9504/10000 (Winner: black) in 57 turns
Game 9505/10000 (Winner: black) in 56 turns
Game 9506/10000 (Winner: black) in 57 turns
Game 9507/10000 (Winner: white) in 44 turns
Game 9508/10000 (Winner: white) in 63 turns
Game 9509/10000 (Winner: white) 

Game 9675/10000 (Winner: black) in 76 turns
Game 9676/10000 (Winner: black) in 53 turns
Game 9677/10000 (Winner: black) in 53 turns
Game 9678/10000 (Winner: white) in 42 turns
Game 9679/10000 (Winner: black) in 68 turns
Game 9680/10000 (Winner: white) in 50 turns
Game 9681/10000 (Winner: black) in 81 turns
Game 9682/10000 (Winner: black) in 62 turns
Game 9683/10000 (Winner: black) in 67 turns
Game 9684/10000 (Winner: black) in 64 turns
Game 9685/10000 (Winner: black) in 51 turns
Game 9686/10000 (Winner: black) in 34 turns
Game 9687/10000 (Winner: black) in 53 turns
Game 9688/10000 (Winner: black) in 50 turns
Game 9689/10000 (Winner: white) in 55 turns
Game 9690/10000 (Winner: black) in 58 turns
Game 9691/10000 (Winner: black) in 46 turns
Game 9692/10000 (Winner: black) in 56 turns
Game 9693/10000 (Winner: black) in 53 turns
Game 9694/10000 (Winner: black) in 55 turns
Game 9695/10000 (Winner: black) in 44 turns
Game 9696/10000 (Winner: white) in 63 turns
Game 9697/10000 (Winner: black) 

Game 9862/10000 (Winner: black) in 55 turns
Game 9863/10000 (Winner: black) in 49 turns
Game 9864/10000 (Winner: white) in 61 turns
Game 9865/10000 (Winner: white) in 56 turns
Game 9866/10000 (Winner: black) in 53 turns
Game 9867/10000 (Winner: black) in 53 turns
Game 9868/10000 (Winner: black) in 43 turns
Game 9869/10000 (Winner: white) in 75 turns
Game 9870/10000 (Winner: white) in 57 turns
Game 9871/10000 (Winner: black) in 42 turns
Game 9872/10000 (Winner: black) in 62 turns
Game 9873/10000 (Winner: black) in 49 turns
Game 9874/10000 (Winner: black) in 51 turns
Game 9875/10000 (Winner: black) in 45 turns
Game 9876/10000 (Winner: black) in 52 turns
Game 9877/10000 (Winner: black) in 61 turns
Game 9878/10000 (Winner: black) in 52 turns
Game 9879/10000 (Winner: black) in 58 turns
Game 9880/10000 (Winner: black) in 61 turns
Game 9881/10000 (Winner: black) in 56 turns
Game 9882/10000 (Winner: black) in 52 turns
Game 9883/10000 (Winner: black) in 52 turns
Game 9884/10000 (Winner: black) 

[Game 20] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 21:0 of 21 games (100.00%)
[Game 21] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 22:0 of 22 games (100.00%)
[Game 22] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 23:0 of 23 games (100.00%)
[Game 23] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 24:0 of 24 games (100.00%)
[Game 24] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 25:0 of 25 games (100.00%)
[Game 25] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 26:0 of 26 games (100.00%)
[Game 26] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 27:0 of 27 games (100.00%)
[Game 27] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 28:0 of 28 games (100.00%)


[Game 86] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 86:1 of 87 games (98.85%)
[Game 87] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 87:1 of 88 games (98.86%)
[Game 88] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 88:1 of 89 games (98.88%)
[Game 89] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 89:1 of 90 games (98.89%)
[Game 90] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 90:1 of 91 games (98.90%)
[Game 91] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 91:1 of 92 games (98.91%)
[Game 92] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 92:1 of 93 games (98.92%)
[Game 93] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 93:1 of 94 games (98.94%)
[Game 94

Auch ohne Selfplay wird 100% Siegsrate gegen random erreicht, jedoch erst nach 8000 statt nach 5000 Spielen

In [30]:
import Player
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = NoSelfplayModel(sess, Player.ValuePlayer('white', Player.singleton), restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TDGammonModel(sess2, restore=True)
        model.test(games = 200, enemyPlayer = Player.ModelPlayer('white', model2))

Restoring checkpoint: checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-524716
INFO:tensorflow:Restoring parameters from checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-524716
Restoring checkpoint: checkpoints/TD-Gammon/checkpoint.ckpt-1593683
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon/checkpoint.ckpt-1593683
[Game 0] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 0:2 of 2 games (0.00%)
[Game 2] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 1:2 of 3 games (33.33%)
[Game 3] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 1:3 of 4 games (25.00%)
[Game 4] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] 

[Game 59] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 12:48 of 60 games (20.00%)
[Game 60] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 12:49 of 61 games (19.67%)
[Game 61] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 12:50 of 62 games (19.35%)
[Game 62] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 12:51 of 63 games (19.05%)
[Game 63] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 13:51 of 64 games (20.31%)
[Game 64] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 13:52 of 65 games (20.00%)
[Game 65] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 14:52 of 66 games (21.21%)
[Game 66] ModelPlayer [NotSelfplayModel[ValuePla

[Game 119] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 27:93 of 120 games (22.50%)
[Game 120] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 27:94 of 121 games (22.31%)
[Game 121] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 27:95 of 122 games (22.13%)
[Game 122] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 27:96 of 123 games (21.95%)
[Game 123] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 28:96 of 124 games (22.58%)
[Game 124] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 29:96 of 125 games (23.20%)
[Game 125] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 29:97 of 126 games (23.02%)
[Game 126] ModelPlayer [NotSelfpla

[Game 179] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 40:140 of 180 games (22.22%)
[Game 180] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 41:140 of 181 games (22.65%)
[Game 181] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 42:140 of 182 games (23.08%)
[Game 182] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 42:141 of 183 games (22.95%)
[Game 183] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 42:142 of 184 games (22.83%)
[Game 184] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 42:143 of 185 games (22.70%)
[Game 185] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 42:144 of 186 games (22.58%)
[Game 186] ModelPlayer [Not

10000 Trainingsspiele weniger machen sich bemerkbar

In [33]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = NoSelfplayModel(sess, Player.ValuePlayer('white', Player.singleton), restore=True)
    model.test(games = 200, enemyPlayer = Player.ValuePlayer('white', Player.singleton))

Restoring checkpoint: checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-524716
INFO:tensorflow:Restoring parameters from checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-524716
[Game 0] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 1:0 of 1 games (100.00%)
[Game 1] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 2:0 of 2 games (100.00%)
[Game 2] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 3:0 of 3 games (100.00%)
[Game 3] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 4:0 of 4 games (100.00%)
[Game 4] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 5:0 of 5 games (100.00%)
[Game 5] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 5:1 of 6 

[Game 59] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 49:11 of 60 games (81.67%)
[Game 60] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 50:11 of 61 games (81.97%)
[Game 61] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 51:11 of 62 games (82.26%)
[Game 62] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 52:11 of 63 games (82.54%)
[Game 63] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 53:11 of 64 games (82.81%)
[Game 64] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 54:11 of 65 games (83.08%)
[Game 65] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 55:11 of 66 games (83.33%)
[Game 66] ModelPlayer [NotSelfplayModel[ValuePla

[Game 119] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 99:21 of 120 games (82.50%)
[Game 120] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 100:21 of 121 games (82.64%)
[Game 121] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 101:21 of 122 games (82.79%)
[Game 122] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 102:21 of 123 games (82.93%)
[Game 123] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 102:22 of 124 games (82.26%)
[Game 124] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 103:22 of 125 games (82.40%)
[Game 125] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 104:22 of 126 games (82.54%)
[Game 126] ModelPlayer [NotS

[Game 179] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 139:41 of 180 games (77.22%)
[Game 180] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 140:41 of 181 games (77.35%)
[Game 181] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 141:41 of 182 games (77.47%)
[Game 182] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 142:41 of 183 games (77.60%)
[Game 183] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 143:41 of 184 games (77.72%)
[Game 184] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 143:42 of 185 games (77.30%)
[Game 185] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 144:42 of 186 games (77.42%)
[Game 186] ModelPlayer [Not

77% Winrate gegen Singleton nach 10000 Spielen, 5% weniger als mit Selfplay

Lassen wir dann doch mal 10000 Spiele mehr trainieren um die gleiche Anzahl wie das normale TD-Gammon zu haben

In [34]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = NoSelfplayModel(sess, Player.ValuePlayer('white', Player.singleton), restore=True)
    model.train(10000,1000)

Restoring checkpoint: checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-524716
INFO:tensorflow:Restoring parameters from checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-524716
Game 0/10000 (Winner: black) in 54 turns
Game 1/10000 (Winner: black) in 53 turns
Game 2/10000 (Winner: black) in 46 turns
Game 3/10000 (Winner: black) in 54 turns
Game 4/10000 (Winner: black) in 65 turns
Game 5/10000 (Winner: black) in 52 turns
Game 6/10000 (Winner: white) in 59 turns
Game 7/10000 (Winner: black) in 60 turns
Game 8/10000 (Winner: white) in 65 turns
Game 9/10000 (Winner: black) in 47 turns
Game 10/10000 (Winner: black) in 52 turns
Game 11/10000 (Winner: black) in 46 turns
Game 12/10000 (Winner: black) in 47 turns
Game 13/10000 (Winner: white) in 57 turns
Game 14/10000 (Winner: white) in 44 turns
Game 15/10000 (Winner: black) in 47 turns
Game 16/10000 (Winner: white) in 39 turns
Game 17/10000 (Winner: black) in 45 turns
Game 18/10000 (Winner: black) in 71 

Game 190/10000 (Winner: black) in 77 turns
Game 191/10000 (Winner: black) in 70 turns
Game 192/10000 (Winner: white) in 51 turns
Game 193/10000 (Winner: black) in 50 turns
Game 194/10000 (Winner: black) in 47 turns
Game 195/10000 (Winner: black) in 56 turns
Game 196/10000 (Winner: black) in 59 turns
Game 197/10000 (Winner: black) in 51 turns
Game 198/10000 (Winner: black) in 47 turns
Game 199/10000 (Winner: black) in 49 turns
Game 200/10000 (Winner: black) in 53 turns
Game 201/10000 (Winner: black) in 54 turns
Game 202/10000 (Winner: black) in 59 turns
Game 203/10000 (Winner: black) in 53 turns
Game 204/10000 (Winner: white) in 73 turns
Game 205/10000 (Winner: black) in 59 turns
Game 206/10000 (Winner: white) in 58 turns
Game 207/10000 (Winner: black) in 50 turns
Game 208/10000 (Winner: black) in 49 turns
Game 209/10000 (Winner: black) in 60 turns
Game 210/10000 (Winner: black) in 60 turns
Game 211/10000 (Winner: white) in 60 turns
Game 212/10000 (Winner: black) in 63 turns
Game 213/10

Game 382/10000 (Winner: black) in 47 turns
Game 383/10000 (Winner: black) in 50 turns
Game 384/10000 (Winner: white) in 68 turns
Game 385/10000 (Winner: black) in 60 turns
Game 386/10000 (Winner: white) in 47 turns
Game 387/10000 (Winner: black) in 39 turns
Game 388/10000 (Winner: black) in 60 turns
Game 389/10000 (Winner: black) in 50 turns
Game 390/10000 (Winner: black) in 54 turns
Game 391/10000 (Winner: black) in 56 turns
Game 392/10000 (Winner: black) in 81 turns
Game 393/10000 (Winner: black) in 66 turns
Game 394/10000 (Winner: white) in 44 turns
Game 395/10000 (Winner: white) in 72 turns
Game 396/10000 (Winner: black) in 75 turns
Game 397/10000 (Winner: black) in 62 turns
Game 398/10000 (Winner: black) in 44 turns
Game 399/10000 (Winner: black) in 46 turns
Game 400/10000 (Winner: black) in 56 turns
Game 401/10000 (Winner: black) in 45 turns
Game 402/10000 (Winner: black) in 47 turns
Game 403/10000 (Winner: black) in 45 turns
Game 404/10000 (Winner: black) in 56 turns
Game 405/10

Game 573/10000 (Winner: white) in 58 turns
Game 574/10000 (Winner: white) in 78 turns
Game 575/10000 (Winner: black) in 53 turns
Game 576/10000 (Winner: black) in 61 turns
Game 577/10000 (Winner: black) in 64 turns
Game 578/10000 (Winner: black) in 46 turns
Game 579/10000 (Winner: white) in 41 turns
Game 580/10000 (Winner: black) in 46 turns
Game 581/10000 (Winner: black) in 53 turns
Game 582/10000 (Winner: white) in 54 turns
Game 583/10000 (Winner: white) in 59 turns
Game 584/10000 (Winner: white) in 81 turns
Game 585/10000 (Winner: black) in 42 turns
Game 586/10000 (Winner: black) in 55 turns
Game 587/10000 (Winner: white) in 58 turns
Game 588/10000 (Winner: white) in 45 turns
Game 589/10000 (Winner: black) in 54 turns
Game 590/10000 (Winner: black) in 57 turns
Game 591/10000 (Winner: black) in 51 turns
Game 592/10000 (Winner: black) in 69 turns
Game 593/10000 (Winner: black) in 58 turns
Game 594/10000 (Winner: white) in 88 turns
Game 595/10000 (Winner: white) in 59 turns
Game 596/10

Game 765/10000 (Winner: white) in 47 turns
Game 766/10000 (Winner: black) in 43 turns
Game 767/10000 (Winner: white) in 67 turns
Game 768/10000 (Winner: black) in 54 turns
Game 769/10000 (Winner: black) in 61 turns
Game 770/10000 (Winner: black) in 63 turns
Game 771/10000 (Winner: black) in 62 turns
Game 772/10000 (Winner: white) in 74 turns
Game 773/10000 (Winner: white) in 46 turns
Game 774/10000 (Winner: black) in 51 turns
Game 775/10000 (Winner: black) in 48 turns
Game 776/10000 (Winner: black) in 59 turns
Game 777/10000 (Winner: white) in 59 turns
Game 778/10000 (Winner: black) in 67 turns
Game 779/10000 (Winner: black) in 69 turns
Game 780/10000 (Winner: black) in 57 turns
Game 781/10000 (Winner: white) in 79 turns
Game 782/10000 (Winner: black) in 53 turns
Game 783/10000 (Winner: black) in 56 turns
Game 784/10000 (Winner: black) in 56 turns
Game 785/10000 (Winner: black) in 46 turns
Game 786/10000 (Winner: black) in 75 turns
Game 787/10000 (Winner: black) in 54 turns
Game 788/10

...

Game 9460/10000 (Winner: black) in 67 turns
Game 9461/10000 (Winner: black) in 65 turns
Game 9462/10000 (Winner: black) in 53 turns
Game 9463/10000 (Winner: black) in 37 turns
Game 9464/10000 (Winner: black) in 73 turns
Game 9465/10000 (Winner: black) in 59 turns
Game 9466/10000 (Winner: black) in 54 turns
Game 9467/10000 (Winner: white) in 51 turns
Game 9468/10000 (Winner: black) in 52 turns
Game 9469/10000 (Winner: white) in 48 turns
Game 9470/10000 (Winner: black) in 70 turns
Game 9471/10000 (Winner: black) in 43 turns
Game 9472/10000 (Winner: black) in 47 turns
Game 9473/10000 (Winner: black) in 44 turns
Game 9474/10000 (Winner: black) in 60 turns
Game 9475/10000 (Winner: white) in 49 turns
Game 9476/10000 (Winner: black) in 59 turns
Game 9477/10000 (Winner: white) in 59 turns
Game 9478/10000 (Winner: white) in 50 turns
Game 9479/10000 (Winner: white) in 60 turns
Game 9480/10000 (Winner: black) in 79 turns
Game 9481/10000 (Winner: black) in 63 turns
Game 9482/10000 (Winner: black) 

Game 9646/10000 (Winner: white) in 50 turns
Game 9647/10000 (Winner: black) in 43 turns
Game 9648/10000 (Winner: black) in 69 turns
Game 9649/10000 (Winner: black) in 51 turns
Game 9650/10000 (Winner: black) in 50 turns
Game 9651/10000 (Winner: black) in 45 turns
Game 9652/10000 (Winner: black) in 58 turns
Game 9653/10000 (Winner: black) in 86 turns
Game 9654/10000 (Winner: black) in 64 turns
Game 9655/10000 (Winner: black) in 72 turns
Game 9656/10000 (Winner: black) in 51 turns
Game 9657/10000 (Winner: white) in 117 turns
Game 9658/10000 (Winner: black) in 59 turns
Game 9659/10000 (Winner: black) in 52 turns
Game 9660/10000 (Winner: black) in 101 turns
Game 9661/10000 (Winner: black) in 67 turns
Game 9662/10000 (Winner: black) in 62 turns
Game 9663/10000 (Winner: black) in 52 turns
Game 9664/10000 (Winner: white) in 64 turns
Game 9665/10000 (Winner: black) in 54 turns
Game 9666/10000 (Winner: white) in 81 turns
Game 9667/10000 (Winner: white) in 57 turns
Game 9668/10000 (Winner: black

Game 9834/10000 (Winner: black) in 46 turns
Game 9835/10000 (Winner: black) in 43 turns
Game 9836/10000 (Winner: black) in 45 turns
Game 9837/10000 (Winner: black) in 52 turns
Game 9838/10000 (Winner: white) in 56 turns
Game 9839/10000 (Winner: black) in 63 turns
Game 9840/10000 (Winner: black) in 56 turns
Game 9841/10000 (Winner: black) in 58 turns
Game 9842/10000 (Winner: black) in 56 turns
Game 9843/10000 (Winner: black) in 45 turns
Game 9844/10000 (Winner: white) in 53 turns
Game 9845/10000 (Winner: black) in 49 turns
Game 9846/10000 (Winner: black) in 57 turns
Game 9847/10000 (Winner: black) in 45 turns
Game 9848/10000 (Winner: black) in 64 turns
Game 9849/10000 (Winner: black) in 58 turns
Game 9850/10000 (Winner: white) in 41 turns
Game 9851/10000 (Winner: black) in 56 turns
Game 9852/10000 (Winner: black) in 50 turns
Game 9853/10000 (Winner: white) in 53 turns
Game 9854/10000 (Winner: black) in 62 turns
Game 9855/10000 (Winner: black) in 54 turns
Game 9856/10000 (Winner: white) 

[Game 8] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 9:0 of 9 games (100.00%)
[Game 9] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 10:0 of 10 games (100.00%)
[Game 10] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 11:0 of 11 games (100.00%)
[Game 11] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 12:0 of 12 games (100.00%)
[Game 12] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 13:0 of 13 games (100.00%)
[Game 13] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 13:1 of 14 games (92.86%)
[Game 14] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 14:1 of 15 games (93.33%)
[Game 15] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 15:1 of 16 games (93.75%)
[Game 1

[Game 77] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 77:1 of 78 games (98.72%)
[Game 78] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 78:1 of 79 games (98.73%)
[Game 79] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 79:1 of 80 games (98.75%)
[Game 80] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 80:1 of 81 games (98.77%)
[Game 81] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 81:1 of 82 games (98.78%)
[Game 82] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 82:1 of 83 games (98.80%)
[Game 83] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 83:1 of 84 games (98.81%)
[Game 84] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs RandomPlayer (white) 84:1 of 85 games (98.82%)
[Game 85

#### NoSelfplay vs singleton

In [35]:
graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = NoSelfplayModel(sess, Player.ValuePlayer('white', Player.singleton), restore=True)
    model.test(games = 200, enemyPlayer = Player.ValuePlayer('white', Player.singleton))

Restoring checkpoint: checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-1109982
INFO:tensorflow:Restoring parameters from checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-1109982
[Game 0] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 1:0 of 1 games (100.00%)
[Game 1] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 1:1 of 2 games (50.00%)
[Game 2] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 2:1 of 3 games (66.67%)
[Game 3] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 3:1 of 4 games (75.00%)
[Game 4] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 4:1 of 5 games (80.00%)
[Game 5] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 5:1 of 6 ga

[Game 59] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 52:8 of 60 games (86.67%)
[Game 60] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 53:8 of 61 games (86.89%)
[Game 61] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 54:8 of 62 games (87.10%)
[Game 62] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 55:8 of 63 games (87.30%)
[Game 63] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 55:9 of 64 games (85.94%)
[Game 64] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 56:9 of 65 games (86.15%)
[Game 65] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 57:9 of 66 games (86.36%)
[Game 66] ModelPlayer [NotSelfplayModel[ValuePlayer [si

[Game 119] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 99:21 of 120 games (82.50%)
[Game 120] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 100:21 of 121 games (82.64%)
[Game 121] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 100:22 of 122 games (81.97%)
[Game 122] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 101:22 of 123 games (82.11%)
[Game 123] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 102:22 of 124 games (82.26%)
[Game 124] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 103:22 of 125 games (82.40%)
[Game 125] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 104:22 of 126 games (82.54%)
[Game 126] ModelPlayer [NotS

[Game 178] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 141:38 of 179 games (78.77%)
[Game 179] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 142:38 of 180 games (78.89%)
[Game 180] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 143:38 of 181 games (79.01%)
[Game 181] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 144:38 of 182 games (79.12%)
[Game 182] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 145:38 of 183 games (79.23%)
[Game 183] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 146:38 of 184 games (79.35%)
[Game 184] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ValuePlayer [singleton] (white) 146:39 of 185 games (78.92%)
[Game 185] ModelPlayer [Not

79%, 3% weniger als normales TD-Gammon

Der finale Test: NoSelfplay vs Selfplay

In [37]:
import Player
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = NoSelfplayModel(sess, Player.ValuePlayer('white', Player.singleton), restore=True)
    graph2 = tf.Graph()
    sess2 = tf.Session(graph=graph2)
    with sess2.as_default(), graph2.as_default():
        model2 = TDGammonModel(sess2, restore=True)
        model.test(games = 200, enemyPlayer = Player.ModelPlayer('white', model2))

Restoring checkpoint: checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-1109982
INFO:tensorflow:Restoring parameters from checkpoints/NotSelfplayModel[ValuePlayer [singleton]]/checkpoint.ckpt-1109982
Restoring checkpoint: checkpoints/TD-Gammon/checkpoint.ckpt-1593683
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon/checkpoint.ckpt-1593683
[Game 0] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 0:2 of 2 games (0.00%)
[Game 2] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 0:3 of 3 games (0.00%)
[Game 3] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 0:4 of 4 games (0.00%)
[Game 4] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] 

[Game 59] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:55 of 60 games (8.33%)
[Game 60] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:56 of 61 games (8.20%)
[Game 61] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:57 of 62 games (8.06%)
[Game 62] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:58 of 63 games (7.94%)
[Game 63] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:59 of 64 games (7.81%)
[Game 64] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:60 of 65 games (7.69%)
[Game 65] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 5:61 of 66 games (7.58%)
[Game 66] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton

[Game 120] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 13:108 of 121 games (10.74%)
[Game 121] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 14:108 of 122 games (11.48%)
[Game 122] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 14:109 of 123 games (11.38%)
[Game 123] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 14:110 of 124 games (11.29%)
[Game 124] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 15:110 of 125 games (12.00%)
[Game 125] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 15:111 of 126 games (11.90%)
[Game 126] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 15:112 of 127 games (11.81%)
[Game 127] ModelPlayer [Not

[Game 179] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 23:157 of 180 games (12.78%)
[Game 180] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 23:158 of 181 games (12.71%)
[Game 181] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 24:158 of 182 games (13.19%)
[Game 182] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 24:159 of 183 games (13.11%)
[Game 183] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 24:160 of 184 games (13.04%)
[Game 184] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 24:161 of 185 games (12.97%)
[Game 185] ModelPlayer [NotSelfplayModel[ValuePlayer [singleton]]] (black) vs ModelPlayer [TD-Gammon] (white) 25:161 of 186 games (13.44%)
[Game 186] ModelPlayer [Not

13% ...


### Fazit

+ Läuft schnell, vor allem am Anfang, da Singleton bereits weiß wie man spielt

- Trainingserfolg geringer
- Keine Chanche gegen klassisches TD-Gammon bei gleicher Anzahl an Trainingsspielen